#### Create a validation set of 50 barns, with FP/NP markers and a visible boundary layer of the bbox

In [ ]:
from ipyleaflet import Map, GeoJSON, WidgetControl, TileLayer, Marker, basemaps, Polygon, Icon
from ipywidgets import HTML, Layout, Button, VBox
import json
import random

# Load your GeoJSON data
with open('../output/final_data.geojson') as f:
    geojson_data = json.load(f)
for i, feature in enumerate(geojson_data['features']):
    feature['properties']['index'] = i

fp_icon = Icon(icon_url='https://cdn-icons-png.flaticon.com/512/190/190446.png', icon_size=[25, 25], icon_anchor=[12, 24])
fn_icon = Icon(icon_url='https://cdn-icons-png.flaticon.com/512/190/190411.png', icon_size=[25, 25], icon_anchor=[12, 24], color='red')

# Generate bbox of 50 barns
def generate_bbox(features):
    attempts = 0
    while attempts < 20:
        random_feature = random.choice(features)
        random_point = random_feature['geometry']['coordinates'][0][0]
        center_lat, center_lon = random_point[1], random_point[0]
        size = 0.01 
        while True:
            lat_min, lat_max = center_lat - size, center_lat + size
            lon_min, lon_max = center_lon - size, center_lon + size
            filtered_features = [
                feature for feature in features if
                lon_min <= feature['geometry']['coordinates'][0][0][0] <= lon_max and
                lat_min <= feature['geometry']['coordinates'][0][0][1] <= lat_max
            ]
            if len(filtered_features) >= 50:
                print(f"BBox coordinates: Lon ({lon_min}, {lon_max}), Lat ({lat_min}, {lat_max})")
                print(f"Total barns in this bbox: {len(filtered_features)}")  
                return filtered_features, (lat_min, lat_max, lon_min, lon_max)
            size += 0.001
        attempts += 1
    raise ValueError("Failed to find a suitable bbox with 100 barns")

filtered_features, bbox_coords = generate_bbox(geojson_data['features'])
false_positives = []
false_negatives = []

# Initialize the map at the center of the bbox
center_lat = (bbox_coords[0] + bbox_coords[1]) / 2
center_lon = (bbox_coords[2] + bbox_coords[3]) / 2
m = Map(
    center=(center_lat, center_lon),
    zoom=12,
    basemap=basemaps.Esri.WorldImagery,
    max_bounds=[(bbox_coords[0], bbox_coords[2]), (bbox_coords[1], bbox_coords[3])]
)

geo_json = GeoJSON(
    data={'type': 'FeatureCollection', 'features': filtered_features},
    style={'color': 'red', 'fillColor': '#ffffff', 'opacity': 1.0, 'fillOpacity': 0.5},
    hover_style={'color': 'green', 'fillOpacity': 0.8}
)

# Setup buttons and info display
fp_button = Button(description='Mark FP', button_style='danger', layout=Layout(width='auto'))
fn_button = Button(description='Mark FN', button_style='success', layout=Layout(width='auto'))
info_display = HTML(value="Select FP or FN and click on the map to place a marker.")

buttons_container = VBox([fp_button, fn_button, info_display])
widget_control = WidgetControl(widget=buttons_container, position='topright')
m.add_control(widget_control)

current_marking = None  # Tracks the current marking type: 'FP' or 'FN'

def set_fp(b):
    global current_marking
    current_marking = 'FP'
    info_display.value = "Marking False Positives. Click on the map."
    geo_json.on_click(handle_fp)

def set_fn(b):
    global current_marking
    current_marking = 'FN'
    info_display.value = "Marking False Negatives. Click on the map."
    m.on_interaction(handle_fn)


def handle_fp(event, feature, **kwargs):

    global m, false_positives
    geometry = feature['geometry']['coordinates'][0][0]
    lon, lat = geometry[:2]
    if current_marking == 'FP':
        # Check if the click is within the bbox before placing a marker
        if bbox_coords[0] <= lat <= bbox_coords[1] and bbox_coords[2] <= lon <= bbox_coords[3]:
            marker = Marker(location=(lat, lon), draggable=False, icon = fp_icon)
            
            false_positives.append(feature['properties']['index'])
            m.add(marker)
            print(feature['properties']['index'])
            print(f"Marked {current_marking} at Latitude: {lat}, Longitude: {lon}")
        
        else:
            info_display.value = "Click is outside the designated area. Try again within the bbox."


def handle_fn(event, **kwargs):
    if kwargs.get('type') == 'click' and current_marking == 'FN':
        latlon = kwargs.get('coordinates')
        lat, lon = latlon[0], latlon[1]
    
    # Check if the click is within the bbox before placing a marker
        if bbox_coords[0] <= lat <= bbox_coords[1] and bbox_coords[2] <= lon <= bbox_coords[3]:
            marker = Marker(location=(lat, lon), draggable=False ,icon = fn_icon)
            false_negatives.append((lat, lon))  # Save coordinates

            lat_rounded = round(lat, 2)
            lon_rounded = round(lon, 2)
            info_display.value = f"Marked {current_marking} at Latitude: {lat_rounded}, Longitude: {lon_rounded}"
            m.add(marker)
            print(f"Marked {current_marking} at Latitude: {lat}, Longitude: {lon}")
                    
        else:
            info_display.value = "Click is outside the designated area. Try again within the bbox."

# Add a visible boundary around the bounding box
bbox_polygon = Polygon(
    locations=[
        (bbox_coords[0], bbox_coords[2]),
        (bbox_coords[0], bbox_coords[3]),
        (bbox_coords[1], bbox_coords[3]),
        (bbox_coords[1], bbox_coords[2]),
        (bbox_coords[0], bbox_coords[2])  # Closing the polygon
    ],
    color="blue",
    fill_color="#ffffff",
    opacity=0.5,
    weight=2
)


fp_button.on_click(set_fp)
fn_button.on_click(set_fn)


m.add_layer(bbox_polygon)

# Add GeoJSON layer and satellite imagery
esri_satellite = TileLayer(
    url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attribution='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'
)
m.add_layer(geo_json)
m.add_layer(esri_satellite)


m


BBox coordinates: Lon (-98.72027248577854, -98.31827248577855), Lat (41.18060481408151, 41.58260481408151)
Total barns in this bbox: 50


Map(center=[41.38160481408151, -98.51927248577854], controls=(ZoomControl(options=['position', 'zoom_in_text',…

2407
Marked FP at Latitude: 41.40595115052142, Longitude: -98.48124896399133
Marked FN at Latitude: 41.380930388318, Longitude: -98.41552734375001
Marked FN at Latitude: 41.40359574314672, Longitude: -98.47320556640626
Marked FN at Latitude: 41.38196080315541, Longitude: -98.46908569335939
Marked FN at Latitude: 41.50137811173191, Longitude: -98.43612670898439
Marked FN at Latitude: 41.46537017728069, Longitude: -98.4210205078125
Marked FN at Latitude: 41.50240661583931, Longitude: -98.46359252929688


#### Save the information of FPs and FNs

In [ ]:
# Save False Positives
with open('../output/false_positives.txt', 'a') as fp_file:
    for i in false_positives:
        fp_file.write(f"{i}, ")

# Save False Negatives
with open('../output/false_negatives.txt', 'a') as fn_file:
    for lat, lon in false_negatives:
        fn_file.write(f"{lat}, {lon}\n")
